# Comment utiliser le serveur MLFlow en local

<img src="https://uploads-ssl.webflow.com/6108e07db6795265f203a636/61f90cbb8c06383f8944720e_ML%20Flow.png" height="360px">


Dans ce notebook, nous allons utiliser la bibliothèque Mlflow pour suivre les expériences de machine learning.

Nous allons :
- Créer un projet Mlflow
- Créer une expérience
- Créer des runs
- Suivre les métriques, les paramètres et les artefacts
- Visualiser les résultats dans l'interface Mlflow
- Enregistrer un modèle puis le charger dans un autre notebook

## Lancer le conteneur

Lancez cette commande depuis l'emplacement ou vous souhaitez sauvegarder les données et artefacts MLFLow pour lancer le conteneur Docker : 

`docker run -d --name mlflow-local-server -v $(pwd)/mlflow-data:/mlflow-data -p 5001:5001 -e PORT=5001 davidscanu/mlflow-server:v1.0`

### Paramètres

- `-p 5001:5001` : changer le numéro de port en utilisant la structure suivante : `-p port-de-host:port-dans-le-container`.
- `-e PORT=5001` : port utilsé par le serveur MLFLow à l'intérieur du container.
- `--name mlflow-local-server` : changer le nom du container.
- `-v $(pwd)/mlflow-data:/mlflow-data` : les données des logs de MLFlow sont stockées dans un dossier `mlflow-data`. Ce flag permet de synchroniser ce dossier avec votre dossier local avec le dossier `/mlflow-data` à l'intérieur du conteneur. `dossier-local:dossier-conteneur`.

## Importation du jeu de données

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Import dataset
df = pd.read_csv("https://julie-2-next-resources.s3.eu-west-3.amazonaws.com/full-stack-full-time/linear-regression-ft/californian-housing-market-ft/california_housing_market.csv")

# X, y split
X = df.drop("MedHouseVal", axis=1)
y = df.MedHouseVal

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Configuration d'un projet Mlflow

- `local_tracking_uri` : URL du serveur MLFLow local.
- `experiment_name` : nom de l'experiment dans lequel nous sauvegardons nos modèles.

In [8]:
local_tracking_uri = "http://localhost:5001"
experiment_name = "california_housing_market_tutorial"
run_name='Run_01'
artifact_path = "model_housing"


Utilisation de `mlflow.autolog()`
- https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.autolog

In [3]:
import mlflow
from mlflow.models import infer_signature

# Connexion à MLflow
mlflow.set_tracking_uri(local_tracking_uri)

# Configuration de l'autolog
# mlflow.sklearn.autolog()
# mlflow.autolog()

# Configuration d'une expérience (création si elle n'existe pas)
mlflow.set_experiment(experiment_name)

# Connexion à une expérience
experiment = mlflow.get_experiment_by_name(experiment_name)

/home/david/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/david/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


## Enregistrement des métriques, paramètres et artefacts

Listes des artefacts que nous souhaitons sauvegarder :
- Modèle
- Input example
- Signature
- Paramètres
- Métriques



In [4]:
from sklearn.preprocessing import  StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# Infer signature : obtention des informations sur les colonnes en entrée
signature = infer_signature(X_train, y_train)

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name=run_name):
    # Pipeline d'entraînement
    model = Pipeline(steps=[
        ("standard_scaler", StandardScaler()),
        ("Regressor",RandomForestRegressor())
    ])

    # Entraînement du modèle
    model.fit(X_train, y_train)

    # # Log d'une metrique
    # mlflow.log_metric("train_score", model.score(X_train, y_train))

    # # Log de plusieurs métriques
    # mlflow.log_metrics({"accuracy": 0.9, "loss": 0.2})

    # # Log des paramètres
    # mlflow.log_params({"epochs": 10, "batch_size": 32})

    # Log du modèle
    mlflow.sklearn.log_model(model,
                            artifact_path = artifact_path,
                            signature = signature,
                            input_example = X_train[:1]
                            )

# Print Scores
print(f"Train score: {model.score(X_train, y_train)}")
print(f"Test score: {model.score(X_test, y_test)}")

Train score: 0.9739205273582824
Test score: 0.8206496527554539


Vous pouvez accéder à l'exécution automatique la plus récente via la fonction `mlflow.last_active_run()`.

In [19]:
autolog_run = mlflow.last_active_run()
autolog_run

<Run: data=<RunData: metrics={'training_mean_absolute_error': 0.12344426937984533,
 'training_mean_squared_error': 0.036479819630337876,
 'training_r2_score': 0.9726552132861959,
 'training_root_mean_squared_error': 0.1909969100020675,
 'training_score': 0.9726552132861959}, params={'Regressor': 'RandomForestRegressor()',
 'Regressor__bootstrap': 'True',
 'Regressor__ccp_alpha': '0.0',
 'Regressor__criterion': 'squared_error',
 'Regressor__max_depth': 'None',
 'Regressor__max_features': '1.0',
 'Regressor__max_leaf_nodes': 'None',
 'Regressor__max_samples': 'None',
 'Regressor__min_impurity_decrease': '0.0',
 'Regressor__min_samples_leaf': '1',
 'Regressor__min_samples_split': '2',
 'Regressor__min_weight_fraction_leaf': '0.0',
 'Regressor__n_estimators': '100',
 'Regressor__n_jobs': 'None',
 'Regressor__oob_score': 'False',
 'Regressor__random_state': 'None',
 'Regressor__verbose': '0',
 'Regressor__warm_start': 'False',
 'memory': 'None',
 'standard_scaler': 'StandardScaler()',
 'sta

### Log séparemment

In [ ]:
# Envoi des métriques
mlflow.log_metrics({"accuracy": 0.9, "loss": 0.2})

In [ ]:
# Envoi des paramètres
mlflow.log_params({"epochs": 10, "batch_size": 32})

In [ ]:
# Envoi des artefacts
mlflow.log_artifact("./house_prices_model.joblib")

In [ ]:
# Fin de l'expérience
mlflow.end_run()

## Charger un modèle enregistré

In [5]:
import sklearn
import mlflow
import pandas as pd

print(f"version de MLFLow : {mlflow.__version__}")
print(f"version de Scikit-Learn : {sklearn.__version__}")

version de MLFLow : 2.7.1
version de Scikit-Learn : 1.3.1


In [6]:
!python3 --version

Python 3.10.6


In [36]:
mlflow.set_tracking_uri(local_tracking_uri)

# logged_model = 'runs:/15914b844341436da2cd153cc11ea834/model'
logged_model = 'runs:/15914b844341436da2cd153cc11ea834/' + artifact_path

# Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model = mlflow.sklearn.load_model(logged_model)

In [29]:
loaded_model.metadata.to_dict()

{'run_id': '15914b844341436da2cd153cc11ea834',
 'artifact_path': 'model_housing',
 'utc_time_created': '2023-11-10 15:08:47.501578',
 'flavors': {'python_function': {'env': {'conda': 'conda.yaml',
    'virtualenv': 'python_env.yaml'},
   'loader_module': 'mlflow.sklearn',
   'model_path': 'model.pkl',
   'predict_fn': 'predict',
   'python_version': '3.10.6'},
  'sklearn': {'code': None,
   'pickled_model': 'model.pkl',
   'serialization_format': 'cloudpickle',
   'sklearn_version': '1.3.1'}},
 'model_uuid': 'a20f556732f7496b87f11a63d5eebfc4',
 'mlflow_version': '2.7.1',
 'signature': {'inputs': '[{"type": "double", "name": "MedInc"}, {"type": "double", "name": "HouseAge"}, {"type": "double", "name": "AveRooms"}, {"type": "double", "name": "AveBedrms"}, {"type": "double", "name": "Population"}, {"type": "double", "name": "AveOccup"}, {"type": "double", "name": "Latitude"}, {"type": "double", "name": "Longitude"}]',
  'outputs': '[{"type": "double", "name": "MedHouseVal"}]',
  'params':

### Prédiction avec le modèle chargé

In [30]:
# Predict on a Pandas DataFrame.
loaded_model.predict(X_test[:1])

array([1.8546301])

In [33]:
y_pred = loaded_model.predict(X_test)

### Performances

In [39]:
mlflow.sklearn._get_metric_name_list()

['accuracy_score',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'auc',
 'average_precision_score',
 'balanced_accuracy_score',
 'calinski_harabasz_score',
 'class_likelihood_ratios',
 'classification_report',
 'cohen_kappa_score',
 'completeness_score',
 'confusion_matrix',
 'consensus_score',
 'coverage_error',
 'd2_tweedie_score',
 'd2_absolute_error_score',
 'd2_pinball_score',
 'dcg_score',
 'davies_bouldin_score',
 'det_curve',
 'euclidean_distances',
 'explained_variance_score',
 'f1_score',
 'fbeta_score',
 'fowlkes_mallows_score',
 'hamming_loss',
 'hinge_loss',
 'homogeneity_completeness_v_measure',
 'homogeneity_score',
 'jaccard_score',
 'label_ranking_average_precision_score',
 'label_ranking_loss',
 'log_loss',
 'nan_euclidean_distances',
 'matthews_corrcoef',
 'max_error',
 'mean_absolute_error',
 'mean_squared_error',
 'mean_squared_log_error',
 'mean_pinball_loss',
 'mean_poisson_deviance',
 'mean_gamma_deviance',
 'mean_tweedie_deviance',
 'median_absolute_e

In [35]:
loaded_model.mean_squared_error(y_test, y_pred)

AttributeError: 'PyFuncModel' object has no attribute 'mean_squared_error'